In [ ]:
#spark.stop()

# News categorization


## I- Modules import

In [ ]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.feature import  IDF, HashingTF,Pipeline
from math import ceil,log2
from pyspark.ml.classification import LogisticRegression,NaiveBayes,LogisticRegressionModel
from pyspark.sql.functions import col,explode,split

## II- Spark context and session creation

In [ ]:
spark = (SparkSession.builder
    .master("spark://node02:7077")
    .appName("NewsCategorisation")
    #.config('spark.driver.cores','4')
    #.config("spark.executorEnv.LD_LIBRARY_PATH", "/home/team1337/.local/easybuild_new/software/Python/3.8.6-GCCcore-10.2.0/lib:/home/team1337/.local/easybuild_new/software/libffi/3.3-GCCcore-10.2.0/lib64:/home/team1337/.local/easybuild_new/software/GMP/6.2.0-GCCcore-10.2.0/lib:/home/team1337/.local/easybuild_new/software/XZ/5.2.5-GCCcore-10.2.0/lib:/home/team1337/.local/easybuild_new/software/SQLite/3.33.0-GCCcore-10.2.0/lib:/home/team1337/.local/easybuild_new/software/Tcl/8.6.10-GCCcore-10.2.0/lib:/home/team1337/.local/easybuild_new/software/libreadline/8.0-GCCcore-10.2.0/lib:/home/team1337/.local/easybuild_new/software/ncurses/6.2-GCCcore-10.2.0/lib:/home/team1337/.local/easybuild_new/software/bzip2/1.0.8-GCCcore-10.2.0/lib:/home/team1337/.local/easybuild_new/software/binutils/2.35-GCCcore-10.2.0/lib:/home/team1337/.local/easybuild_new/software/zlib/1.2.11-GCCcore-10.2.0/lib:/home/team1337/.local/easybuild_new/software/GCCcore/10.2.0/lib64") \
    #.config("spark.pyspark.python", "/home/team1337/.local/easybuild_new/software/Python/3.8.6-GCCcore-10.2.0/bin/python3") \
    .getOrCreate()
        )
spark

In [ ]:
# Load data
#df = spark.read.csv("input/news.csv", header=True, inferSchema=True)
df = spark.read.parquet("input/news.parquet", header=True, inferSchema=True)

In [ ]:
df.rdd.getNumPartitions()

9

In [ ]:
num_partitions=5*40
df= df.repartition(num_partitions).cache()

In [ ]:
df.rdd.getNumPartitions()

200

In [ ]:
df=df.cache()

24/06/04 19:20:53 WARN CacheManager: Asked to cache already cached data.


In [ ]:
df.count()

1716608

In [ ]:
df.show()

+--------------+--------------------+
|category_label|description_filtered|
+--------------+--------------------+
|          10.0|every bozo need k...|
|          11.0|best redness past...|
|          10.0|fashion show used...|
|           9.0|challenge present...|
|          10.0|man vintage show ...|
|          11.0|hidden mickey spo...|
|          10.0|next fashion uppe...|
|          11.0|work home rabbi d...|
|           9.0|mommy manage nt g...|
|           9.0|ontogenesis hormo...|
|          10.0|7 gross grooming ...|
|           9.0|period display bo...|
|          10.0|transformation ph...|
|          10.0|morena baccarin g...|
|          11.0|expat recovery ro...|
|          11.0|amsterdam diverse...|
|          11.0|view afar make ma...|
|          10.0|new house york fa...|
|           9.0|5 dumbest affair ...|
|           9.0|   valentine day kid|
+--------------+--------------------+
only showing top 20 rows



In [ ]:
# Create a new DataFrame with description_filtered as arrays
df= df.withColumn('description_filtered', split(col('description_filtered'), ' '))
# Show the new DataFrame
df.show(truncate=False)

+--------------+----------------------------------------------------------------------------------------------------------------+
|category_label|description_filtered                                                                                            |
+--------------+----------------------------------------------------------------------------------------------------------------+
|10.0          |[every, bozo, need, know, fashion]                                                                              |
|11.0          |[best, redness, pasta, sauce, italian, love, life, ve, never, heard]                                            |
|10.0          |[fashion, show, used, represent, raucous, liaison, video]                                                       |
|9.0           |[challenge, present, girl, constitute, dying]                                                                   |
|10.0          |[man, vintage, show, delivers, carefully, curated, habiliment, culled, aro

In [ ]:
exploded_df=df.select(explode(df.description_filtered)).alias('words')

In [ ]:
#df=df.unpersist()

In [ ]:
unique_words=exploded_df.distinct()

In [ ]:
unique_words=unique_words.cache()

In [ ]:
unique_words.show()

+--------+
|     col|
+--------+
|     art|
|   oscar|
|newlywed|
|  harder|
|  teigen|
|   still|
|   monte|
|   carlo|
| hitched|
|  filing|
|    pant|
|  travel|
|  bazaar|
|  poetry|
|  outfit|
|   1970s|
| elevate|
|  online|
| embrace|
|  patton|
+--------+
only showing top 20 rows



In [ ]:
vocabulary_size=unique_words.count()
vocabulary_size

128622

In [ ]:
unique_words=unique_words.unpersist()

In [ ]:
n=ceil(log2(vocabulary_size))
n

17

In [ ]:
# Function to evaluate model and get best parameters
def evaluate_model(model, model_name):
    print('Training the model')

    # Train the model using cross-validation
    fitted_model = model.fit(train_set)

    # Get the best model from cross-validation
    best_model = fitted_model.stages[-1].bestModel

    print('Making predictions on the training set')
    # Make predictions on the training set
    train_predictions = fitted_model.transform(train_set)

    print('Making predictions on the test set')
    # Make predictions on the test set
    test_predictions = fitted_model.transform(test_set)

    # Initialize the evaluator
    evaluator = MulticlassClassificationEvaluator(labelCol="category_label", predictionCol="prediction", metricName="accuracy")

    print('Evaluating the model on training set')
    # Evaluate the model on the training set
    train_accuracy = evaluator.evaluate(train_predictions)

    print('Evaluating the model on test set')
    # Evaluate the model on the test set
    test_accuracy = evaluator.evaluate(test_predictions)

    print(f"{model_name} Train Accuracy: {train_accuracy}")
    print(f"{model_name} Test Accuracy: {test_accuracy}")

    # Print the best parameters
    print(f"Best parameters for {model_name}:")

    for param, value in best_model.extractParamMap().items():
        print(f"  {param.name}: {value}")

    return train_accuracy, test_accuracy,best_model

In [ ]:
# Define the HashingTF and IDF stages
hashingTF = HashingTF(inputCol="description_filtered", outputCol="rawFeatures", numFeatures=2**n)
idf = IDF(inputCol="rawFeatures", outputCol="features")

# Define the classifiers
lr = LogisticRegression(labelCol="category_label", featuresCol="features")

nb = NaiveBayes(labelCol="category_label", featuresCol="features")

# Define parameter grids
paramGrid_nb = ParamGridBuilder() \
    .addGrid(nb.smoothing, [0.5, 1.0, 2.0]) \
    .build()



reg_values = np.logspace(-4, 4, num=100)

l1_ratios = np.linspace(0, 1, num=10)

paramGrid_lr = (ParamGridBuilder()
    .addGrid(lr.regParam, reg_values)
    #.addGrid(lr.elasticNetParam, l1_ratios)
    .build())



paramGrid_dt = (ParamGridBuilder()
    #.addGrid(dt.numTrees, [10, 50, 100])
    .addGrid(dt.maxDepth, [5, 10, 20])
    .build())

# Create cross validators
cv_nb = CrossValidator(estimator=nb, estimatorParamMaps=ParamGridBuilder().build(), #paramGrid_nb,
                       evaluator=MulticlassClassificationEvaluator(labelCol="category_label", predictionCol="prediction", metricName="accuracy"),
                       numFolds=3, parallelism=1)

cv_lr = CrossValidator(estimator=lr, estimatorParamMaps=ParamGridBuilder().build(), #paramGrid_lr,
                       evaluator=MulticlassClassificationEvaluator(labelCol="category_label", predictionCol="prediction", metricName="accuracy"),
                       numFolds=3, parallelism=1)

cv_dt = CrossValidator(estimator=dt, estimatorParamMaps=ParamGridBuilder().build(), # paramGrid_rf,
                       evaluator=MulticlassClassificationEvaluator(labelCol="category_label", predictionCol="prediction", metricName="accuracy"),
                       numFolds=3,parallelism=3)

# Create pipelines
pipeline_nb = Pipeline(stages=[hashingTF, idf, cv_nb])
pipeline_lr = Pipeline(stages=[hashingTF, idf, cv_lr])

# Split data
(train_set, test_set) = df.randomSplit([0.8, 0.2], seed=0)



# Evaluate models using the evaluate_model function
results = {}
for model_pipeline, model_name in [
    (pipeline_nb, "Naive Bayes"),
    #(pipeline_lr, "Logistic Regression"),

]:
    print(f'Evaluating {model_name}')
    train_accuracy, test_accuracy,best_model = evaluate_model(model_pipeline, model_name)
    results[model_name] = {"train_accuracy": train_accuracy, "test_accuracy": test_accuracy}



Evaluating Naive Bayes
Training the model


24/06/04 19:56:22 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
24/06/04 19:56:37 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
24/06/04 19:56:39 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
24/06/04 19:56:44 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
24/06/04 19:56:45 WARN DAGScheduler: Broadcasting large task binary with size 34.1 MiB
24/06/04 19:56:47 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
24/06/04 19:56:48 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
24/06/04 19:56:51 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
24/06/04 19:56:55 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
24/06/04 19:56:57 WARN DAGScheduler: Broadcasting large task binary with size 34.1 MiB
24/06/04 19:56:58 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
24/06/04 19:56:59 WARN DAGScheduler: Broadcasting la

Making predictions on the training set
Making predictions on the test set
Evaluating the model on training set


24/06/04 19:57:17 WARN DAGScheduler: Broadcasting large task binary with size 34.1 MiB


Evaluating the model on test set


24/06/04 19:57:23 WARN DAGScheduler: Broadcasting large task binary with size 34.1 MiB


Naive Bayes Train Accuracy: 0.786215323314107
Naive Bayes Test Accuracy: 0.7531130303074436
Best parameters for Naive Bayes:
  featuresCol: features
  labelCol: category_label
  modelType: multinomial
  predictionCol: prediction
  probabilityCol: probability
  rawPredictionCol: rawPrediction
  smoothing: 1.0


In [ ]:
best_model.save('pp')

24/06/04 20:03:43 WARN TaskSetManager: Stage 216 contains a task of very large size (33450 KiB). The maximum recommended task size is 1000 KiB.


In [ ]:
test_set=te
loaded_model = NaiveBayesModel.load('pp')

24/06/04 20:31:50 ERROR Instrumentation: java.lang.IllegalArgumentException: requirement failed: Error loading metadata: Expected class name org.apache.spark.ml.PipelineModel but found class name org.apache.spark.ml.classification.NaiveBayesModel
	at scala.Predef$.require(Predef.scala:281)
	at org.apache.spark.ml.util.DefaultParamsReader$.parseMetadata(ReadWrite.scala:610)
	at org.apache.spark.ml.util.DefaultParamsReader$.loadMetadata(ReadWrite.scala:588)
	at org.apache.spark.ml.Pipeline$SharedReadWrite$.$anonfun$load$3(Pipeline.scala:269)
	at org.apache.spark.ml.util.Instrumentation$.$anonfun$instrumented$1(Instrumentation.scala:191)
	at scala.util.Try$.apply(Try.scala:213)
	at org.apache.spark.ml.util.Instrumentation$.instrumented(Instrumentation.scala:191)
	at org.apache.spark.ml.Pipeline$SharedReadWrite$.load(Pipeline.scala:268)
	at org.apache.spark.ml.PipelineModel$PipelineModelReader.$anonfun$load$7(Pipeline.scala:356)
	at org.apache.spark.ml.MLEvents.withLoadInstanceEvent(events

IllegalArgumentException: requirement failed: Error loading metadata: Expected class name org.apache.spark.ml.PipelineModel but found class name org.apache.spark.ml.classification.NaiveBayesModel

In [ ]:
n = 17  # Assuming n is defined somewhere
hashingTF = HashingTF(inputCol="description_filtered", outputCol="rawFeatures", numFeatures=2**n)
idf = IDF(inputCol="rawFeatures", outputCol="features")

# Apply HashingTF to the test set
test_set_with_raw_features = hashingTF.transform(test_set)

# Fit IDF on the training set to get the IDF model
idf_model = idf.fit(test_set_with_raw_features)

# Apply IDF to the test set
test_set_with_features = idf_model.transform(test_set_with_raw_features)

# Show the test set with the new features column
test_set_with_features.select("description_filtered", "rawFeatures", "features").show(5, truncate=False)

In [ ]:
loaded_model=NaiveBayesModel.load('pp')
# Verify the model is loaded by checking its parameters or making predictions
print("Loaded model parameters:")
for param, value in loaded_model.extractParamMap().items():
    print(f"  {param.name}: {value}")

# Making predictions on a new dataset
# Assuming test_set is your test DataFrame
predictions = loaded_model.transform(test_set_with_features)

Loaded model parameters:
  featuresCol: features
  labelCol: category_label
  modelType: multinomial
  predictionCol: prediction
  probabilityCol: probability
  rawPredictionCol: rawPrediction
  smoothing: 1.0


In [ ]:
predictions.select('description_filtered','prediction').show()

24/06/04 20:45:14 WARN DAGScheduler: Broadcasting large task binary with size 34.1 MiB


+--------------------+----------+
|description_filtered|prediction|
+--------------------+----------+
|[apple, watch, ev...|      26.0|
|[arthveda, fund, ...|       0.0|
|[bank, india, hea...|       0.0|
|[bank, looking, a...|       0.0|
|[bank, may, accel...|       0.0|
|[better, liquidit...|       0.0|
|[bharti, axa, lif...|       0.0|
|[bill, seek, repl...|       0.0|
|[central, bank, a...|       0.0|
|[cleanup, exercis...|       0.0|
|[court, toss, 21,...|      27.0|
|[credit, growth, ...|       0.0|
|[credit, rating, ...|       0.0|
|[deal, ergos, sta...|       0.0|
|[describes, brexi...|       2.0|
|[draftkings, repo...|       0.0|
|[farmer, longer, ...|       0.0|
|[financial, intel...|       0.0|
|[firstquarter, ad...|       0.0|
|[given, poor, fis...|       0.0|
+--------------------+----------+
only showing top 20 rows



In [ ]:
#df.unpersist()